# 0 Setting

In [145]:
# Parameter Setting
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
folder_name = 'Research-(D5) Synthesized input model'

config = {
    'learning_rate': 2e-4,
    'batch_size': 32,
    'seq_length': 1,

    'shuffle': False,
    'criterion': torch.nn.CrossEntropyLoss(),
    'seed': 42,
    'valid_ratio': 0.2,
    'test_ratio': 0.2,
    'max_length': 512,
    'n_epochs': 3000,
    'early_stop': 100,
}

feature = [
    # X_1
    # 'input_ids',
    # 'attention_mask',
    # 'section_dummy',
    # 'publication_dummy',

    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'today_return_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    'sentiment',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',

    # y
    # '1_day_return',
    # '2_day_return',
    # '3_day_return',
    # '4_day_return',
    # '5_day_return',
    # '1_day_return_cate',
    # '2_day_return_cate',
    # '3_day_return_cate',
    # '4_day_return_cate',
    # '5_day_return_cate',
    # '^DJI', '^DJI_1_day_return', '^GSPC', '^GSPC_1_day_return',
    #    '^NDX', '^NDX_1_day_return', '^IXIC', '^IXIC_1_day_return', '^SOX',
    #    '^SOX_1_day_return',
    # 'excess_return_^DJI',
    # 'excess_return_^DJI_cate',
    # 'excess_return_^GSPC',
    'excess_return_^GSPC_cate',
    # 'excess_return_^NDX',
    # 'excess_return_^NDX_cate',
    # 'excess_return_^IXIC',
    # 'excess_return_^IXIC_cate',
    # 'excess_return_^SOX',
    # 'excess_return_^SOX_cate',


    # Do not mark the datetime, it's for operation
    'datetime',
    ]

# All the news dataset
# time_start = '2016-01-01T00:00:00'
# time_end = '2020-04-02T00:00:00'

time_start = '2016-01-01T00:00:00'
time_end = '2019-12-31T00:00:00'

print(len(feature)-2)

company_list = [
    "Information Technology",
    "Health Care",
    "Financials",
    "Industrials",
    "Consumer Discretionary",
    "Energy",
    "Materials",
    "Communication Services",
    "Utilities",
    "Real Estate",
    "Consumer Staples"
]

process_id = 0  #26

company_name = company_list[process_id]
config_2 = {'input_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/data/2_'+company_name+'_for_model.csv',
            'save_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model.ckpt',
            }

7


## (1) Import

In [146]:
# Google
from google.colab import drive
drive.mount('/content/drive')

# pip installation
!pip install transformers

# Basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# PyTorch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from transformers import XLNetModel, XLNetTokenizer, BertTokenizer, BertModel

# others
from datetime import datetime, timedelta
from tqdm import tqdm
from torchsummary import summary
import ast

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed for reproducibility
same_seed(config['seed'])


In [148]:
df = pd.read_csv(config_2['input_path'])
df = df.sort_values(by='datetime', ascending=True)
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,today_return,today_return_cate,...,excess_return_^IXIC_cate,excess_return_^SOX,excess_return_^SOX_cate,excess_return_^NYA,excess_return_^NYA_cate,input_ids,attention_mask,section_dummy,publication_dummy,sentiment
0,38.157610,38.357196,37.749361,38.348125,21638600,0.0,0.0,0.0,0.004993,1,...,0,0.006342,1,-0.007764,0,"[[101, 18817, 1005, 1055, 8292, 2015, 2355, 25...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, ...",0.486591
1,38.466063,38.556784,38.075959,38.248329,16067200,0.0,0.0,0.0,-0.005660,0,...,0,0.013500,1,0.006355,1,"[[101, 13938, 1005, 1055, 8292, 2015, 2355, 25...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, ...",0.479906
2,37.685864,38.030605,37.549780,37.776585,13858400,0.0,0.0,0.0,0.002407,1,...,1,0.001351,1,0.001034,1,"[[101, 2115, 9432, 27918, 1024, 3795, 15768, 1...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, ...",0.480993
3,37.005450,37.486274,36.642561,36.660706,16840700,0.0,0.0,0.0,-0.009316,0,...,1,0.003363,1,0.000485,1,"[[101, 1996, 3316, 2008, 3627, 4684, 2024, 263...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, ...",0.482691
4,36.978211,37.105225,36.325016,36.370377,19233100,0.0,0.0,0.0,-0.016438,0,...,1,0.003958,1,0.004467,1,"[[101, 15768, 2485, 3816, 13463, 3514, 25912, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, ...",0.479977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,77.318843,78.772933,75.855056,76.184647,29242700,0.0,0.0,0.0,-0.014669,0,...,1,0.002589,1,0.009174,1,"[[101, 6207, 7283, 5599, 7276, 5126, 2147, 200...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, ...",0.484674
1066,77.202517,79.577528,76.960168,79.393349,23740100,0.0,0.0,0.0,0.028378,1,...,0,0.004554,1,-0.008134,0,"[[101, 2833, 4425, 15508, 6617, 1999, 2149, 20...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",0.478539
1067,79.209155,80.634165,77.503021,77.910172,19994400,0.0,0.0,0.0,-0.016399,0,...,0,0.005237,1,-0.006226,0,"[[101, 21887, 23350, 20861, 1999, 8244, 1521, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, ...",0.503434
1068,75.215239,76.824436,73.644826,74.197380,27492400,0.0,0.0,0.0,-0.013533,0,...,1,-0.013970,0,0.001834,1,"[[101, 21887, 23350, 2444, 14409, 1024, 1057, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, ...",0.354164


In [149]:
df.publication_dummy[0]

'[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]'

In [150]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Capital Gains', 'today_return', 'today_return_cate', '1_day_return',
       '2_day_return', '3_day_return', '4_day_return', '5_day_return',
       '1_day_return_cate', '2_day_return_cate', '3_day_return_cate',
       '4_day_return_cate', '5_day_return_cate', 'Sma', 'Rsi', 'Kd', 'Ema_12',
       'Ema_26', 'Macd', 'datetime', '^DJI', '^DJI_1_day_return', '^GSPC',
       '^GSPC_1_day_return', '^NDX', '^NDX_1_day_return', '^IXIC',
       '^IXIC_1_day_return', '^SOX', '^SOX_1_day_return', '^NYA',
       '^NYA_1_day_return', 'excess_return_^DJI', 'excess_return_^DJI_cate',
       'excess_return_^GSPC', 'excess_return_^GSPC_cate', 'excess_return_^NDX',
       'excess_return_^NDX_cate', 'excess_return_^IXIC',
       'excess_return_^IXIC_cate', 'excess_return_^SOX',
       'excess_return_^SOX_cate', 'excess_return_^NYA',
       'excess_return_^NYA_cate', 'input_ids', 'attention_mask',
       'section_dummy', '

In [151]:
# Only contain selected features
df = df[feature]
df.columns

Index(['today_return', 'sentiment', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX',
       'excess_return_^GSPC_cate', 'datetime'],
      dtype='object')

## check data

In [152]:
df.isnull().sum()

today_return                0
sentiment                   0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
excess_return_^GSPC_cate    0
datetime                    0
dtype: int64

In [153]:
df = df.dropna()
df = df.reset_index(drop=True)
df.isnull().sum()

today_return                0
sentiment                   0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
excess_return_^GSPC_cate    0
datetime                    0
dtype: int64

## (2) Time Period Selection

In [154]:
# We use index to filter for time periods
df = df[(df['datetime']> time_start) & (df['datetime'] < time_end)]

# Drop datetime after using it
df.drop(columns=['datetime'], inplace=True)
df.shape

(1006, 8)

## (3) Transform str back to list

In [155]:
df.isnull().sum()

today_return                0
sentiment                   0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
excess_return_^GSPC_cate    0
dtype: int64

In [156]:
# # 将字符串转换回列表的函数
# def string_to_list(s):
#     return ast.literal_eval(s)

# # 将列中的字符串转换回列表
# df['input_ids'] = df['input_ids'].apply(string_to_list)
# df['attention_mask'] = df['attention_mask'].apply(string_to_list)
# df['section_dummy'] = df['section_dummy'].apply(string_to_list)
# df['publication_dummy'] = df['publication_dummy'].apply(string_to_list)

## (4) Train_test_split

In [157]:
# 1. Set up X, y
to_remove_list = ['datetime', 'excess_return_^GSPC_cate']

# Filter out values in to_remove_list
filtered_list = [x for x in feature if x not in to_remove_list]

X = df[filtered_list]
y = df['excess_return_^GSPC_cate']



# # 1. Set up X, y
# to_remove_list = ['datetime', '1_day_return_cate']

# # Filter out values in to_remove_list
# filtered_list = [x for x in feature if x not in to_remove_list]

# X = df[filtered_list]
# y = df['1_day_return_cate']

In [158]:
# print(X['section_dummy'])
# print(type(X['section_dummy'][0]))
# temp_array = np.array(X['section_dummy'][0])
# print(temp_array.dtype)

# def int_to_float(int_list):
#   float_list = np.array(int_list, dtype=np.float32)
#   return float_list
# X['section_dummy'] = X['section_dummy'].apply(int_to_float)
# X['publication_dummy'] = X['publication_dummy'].apply(int_to_float)\
# temp_array = np.array(X['section_dummy'][0])
# print(temp_array.dtype)


In [159]:
# def int_to_float(int_list):
#   float_list = np.array(int_list, dtype=np.float32)
#   return float_list
# X['section_dummy'] = X['section_dummy'].apply(int_to_float)
# X['publication_dummy'] = X['publication_dummy'].apply(int_to_float)

In [160]:
# Check X, y shape
print('X:', X.shape)
print('y:', y.shape)

X: (1006, 7)
y: (1006,)


In [161]:
# 2. train_test_split
# val dataset for final examination

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=config['seed'], shuffle=config['shuffle'])
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=config['seed'], shuffle=config['shuffle'])
X_train


,today_return,sentiment,^DJI,^GSPC,^NDX,^IXIC,^SOX
0,0.004993,0.486591,-0.014739,-0.012531,0.002854,0.001111,0.008979
1,-0.005660,0.479906,0.000651,0.001455,-0.006175,-0.005370,-0.012003
2,0.002407,0.480993,-0.014475,-0.010663,0.007758,0.004570,-0.011092
3,-0.009316,0.482691,-0.022161,-0.021271,-0.009984,-0.009917,-0.010667
4,-0.016438,0.479977,-0.010456,-0.012302,-0.015069,-0.016601,-0.019801
...,...,...,...,...,...,...,...
598,-0.000870,0.484560,0.000298,-0.001612,-0.001512,-0.001358,-0.003253
599,-0.000432,0.545957,0.005234,-0.000870,-0.002137,-0.001661,-0.004453
600,-0.006303,0.494470,-0.008509,-0.005076,-0.005924,-0.005712,-0.001986
601,0.015698,0.490091,0.005215,0.007115,0.015533,0.012397,0.015664


## (5) Scaler

In [162]:
scale_feature = [
    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'Today_trend_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    'sentiment',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',
    # 'datetime'
    ]

def CustomScaler(X_train, X_val, X_test):
  scaler = MinMaxScaler()
  for i in scale_feature:

    # 對特定欄位進行標準化
    X_train_scaled = scaler.fit_transform(X_train[[i]])
    X_val_scaled = scaler.transform(X_val[[i]])
    X_test_scaled = scaler.transform(X_test[[i]])

    # 將標準化後的值重新賦值給 DataFrame
    X_train[i] = X_train_scaled
    X_val[i] = X_val_scaled
    X_test[i] = X_test_scaled

  return X_train, X_val, X_test

X_train, X_val, X_test = CustomScaler(X_train, X_val, X_test)

X_train

,today_return,sentiment,^DJI,^GSPC,^NDX,^IXIC,^SOX
0,0.564830,0.585631,0.355624,0.377236,0.525722,0.515721,0.591670
1,0.442231,0.545035,0.557596,0.578553,0.424253,0.436452,0.374338
2,0.535075,0.551635,0.359087,0.404126,0.580837,0.558035,0.383766
3,0.400162,0.561947,0.258224,0.251419,0.381444,0.380842,0.388167
4,0.318206,0.545467,0.411837,0.380522,0.324291,0.299087,0.293563
...,...,...,...,...,...,...,...
598,0.497355,0.573301,0.552968,0.534407,0.476651,0.485527,0.464971
599,0.502404,0.946136,0.617738,0.545086,0.469628,0.481824,0.452541
600,0.434839,0.633477,0.437380,0.484541,0.427071,0.432268,0.478096
601,0.688021,0.606886,0.617488,0.660028,0.668223,0.653762,0.660923


## (6) Check number

In [163]:
def calculate_class_stats(y):
    class_counts = y.value_counts()
    total_samples = len(y)
    class_ratios = class_counts / total_samples
    return class_counts, class_ratios

# 計算類別數量和比例
train_class_counts, train_class_ratios = calculate_class_stats(y_train)
val_class_counts, val_class_ratios = calculate_class_stats(y_val)
test_class_counts, test_class_ratios = calculate_class_stats(y_test)

# 創建包含數量和比例的 DataFrame
class_stats = pd.DataFrame({
    'Train Count': train_class_counts,
    'Train Ratio': train_class_ratios,
    'Validation Count': val_class_counts,
    'Validation Ratio': val_class_ratios,
    'Test Count': test_class_counts,
    'Test Ratio': test_class_ratios
})

# 打印 DataFrame
print(class_stats)


   Train Count  Train Ratio  Validation Count  Validation Ratio  Test Count  \
1          322     0.533997               113          0.562189         114   
0          281     0.466003                88          0.437811          88   

   Test Ratio  
1    0.564356  
0    0.435644  


In [164]:
# Time period
print('Time Period')
print('From:', time_start)
print('To:', time_end, '\n')

# Sample size
print('Sample size:', X.shape[0])
print('Feature:', X.columns, '\n')
print('Target:', y.name, '\n')
print('Train: Val: Test = 0.81: 0.1: 0.09=', X_train.shape[0], X_test.shape[0], X_val.shape[0])

Time Period
From: 2016-01-01T00:00:00
To: 2019-12-31T00:00:00 

Sample size: 1006
Feature: Index(['today_return', 'sentiment', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX'], dtype='object') 

Target: excess_return_^GSPC_cate 

Train: Val: Test = 0.81: 0.1: 0.09= 603 202 201


# Model

## (1) Dataset & Dataloader

In [165]:
# Dataset
# X_1 =['input_ids', 'attention_mask', 'section_dummy', 'publication_dummy']


class CustomDataset(Dataset):
    def __init__(self, X, y, config):
        # X_2
        self.X_2 = torch.tensor(X.values, dtype=torch.float)

        # y
        self.y = torch.tensor(y.values, dtype=torch.long)

        # other setting
        self.len = X.shape[0]
        self.seq_length = config['seq_length']

    def __getitem__(self,idx):
        # X_2
        X_2 = self.X_2[idx : idx + self.seq_length]

        # 3. y
        y = self.y[idx + self.seq_length - 1]

        return X_2, y

    def __len__(self):
        return self.len - self.seq_length

In [166]:
# DataLoader
train_dataset = CustomDataset(X_train, y_train, config)
val_dataset = CustomDataset(X_val, y_val, config)
test_dataset = CustomDataset(X_test, y_test, config)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], pin_memory=True)

In [167]:
# Check loader output
for batch in train_loader:
    X_2, y = batch

    # 打印批次数据的形状，以确保它们符合预期
    # print("Input IDs shape:", input_ids.shape)
    # print("Attention Mask shape:", attention_mask.shape)
    # print("Section shape:", section.shape)
    # print("Publication shape:", publication.shape)
    print("X_2 shape:", X_2.shape)
    print("Labels shape:", y.shape)

    # print("Input IDs:", input_ids)
    # print("Attention Mask:", attention_mask)
    # print("Section:", section)
    # print("Publication:", publication)
    print("X_2:", X_2)
    print("Labels:", y)

    break


X_2 shape: torch.Size([32, 1, 7])
Labels shape: torch.Size([32])
X_2: tensor([[[0.5648, 0.5856, 0.3556, 0.3772, 0.5257, 0.5157, 0.5917]],

        [[0.4422, 0.5450, 0.5576, 0.5786, 0.4243, 0.4365, 0.3743]],

        [[0.5351, 0.5516, 0.3591, 0.4041, 0.5808, 0.5580, 0.3838]],

        [[0.4002, 0.5619, 0.2582, 0.2514, 0.3814, 0.3808, 0.3882]],

        [[0.3182, 0.5455, 0.4118, 0.3805, 0.3243, 0.2991, 0.2936]],

        [[0.5017, 0.5968, 0.5810, 0.5393, 0.4481, 0.4094, 0.4525]],

        [[0.5215, 0.5467, 0.6267, 0.6386, 0.5221, 0.5136, 0.4688]],

        [[0.1157, 0.5465, 0.2511, 0.1862, 0.0559, 0.0344, 0.1788]],

        [[0.6747, 0.5400, 0.7278, 0.7871, 0.6856, 0.6895, 0.6631]],

        [[0.5603, 0.5940, 0.2552, 0.2846, 0.5406, 0.5680, 0.4853]],

        [[0.3859, 0.5604, 0.5544, 0.5017, 0.3532, 0.3109, 0.3088]],

        [[0.6144, 0.5788, 0.3663, 0.4283, 0.6536, 0.6867, 0.6309]],

        [[0.5074, 0.5797, 0.6438, 0.6158, 0.4832, 0.4786, 0.5401]],

        [[0.6051, 0.6183, 0.6912,

## Focal Loss

In [168]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class FocalLoss(nn.Module):
#     def __init__(self, gamma=2, alpha=1):
#         super(FocalLoss, self).__init__()
#         self.gamma = gamma
#         self.alpha = alpha

#     def forward(self, input, target):
#         # 计算交叉熵损失
#         ce_loss = F.cross_entropy(input, target, reduction='none')
#         pt = torch.exp(-ce_loss)
#         focal_loss = (1 - pt) ** self.gamma * ce_loss

#         # 应用类别权重（alpha）
#         if self.alpha is not None:
#             alpha = self.alpha
#             focal_loss = alpha * focal_loss

#         return torch.mean(focal_loss)

# # 创建一个示例模型和示例目标
# model = nn.Linear(10, 2)  # 示例模型
# target = torch.randint(0, 2, (5,))  # 示例目标，0或1

# # 创建Focal Loss函数的实例
# criterion = FocalLoss(gamma=2, alpha=1)

# # 计算损失
# output = model(torch.randn(5, 10))
# loss = criterion(output, target)

# print("Focal Loss值：", loss.item())


In [169]:
# from torch import nn
# import torch
# from torch.nn import functional as F

# class focal_loss(nn.Module):
#     def __init__(self, alpha=None, gamma=2, num_classes = 3, size_average=True):
#         super(focal_loss,self).__init__()
#         self.size_average = size_average
#         if alpha is None:
#             self.alpha = torch.ones(num_classes)
#         elif isinstance(alpha,list):
#             assert len(alpha)==num_classes   # α可以以list方式输入,size:[num_classes] 用于对不同类别精细地赋予权重
#             self.alpha = torch.Tensor(alpha)
#         else:
#             assert alpha<1   #如果α为一个常数,则降低第一类的影响,在目标检测中第一类为背景类
#             self.alpha = torch.zeros(num_classes)
#             self.alpha[0] += alpha
#             self.alpha[1:] += (1-alpha) # α 最终为 [ α, 1-α, 1-α, 1-α, 1-α, ...] size:[num_classes]

#         self.gamma = gamma

#         print('Focal Loss:')
#         print('    Alpha = {}'.format(self.alpha))
#         print('    Gamma = {}'.format(self.gamma))

#     def forward(self, preds, labels):

#         preds = preds.view(-1,preds.size(-1))
#         alpha = self.alpha.to(preds.device)
#         preds_logsoft = F.log_softmax(preds, dim=1) # log_softmax
#         preds_softmax = torch.exp(preds_logsoft)    # softmax

#         preds_softmax = preds_softmax.gather(1,labels.view(-1,1))   # 这部分实现nll_loss ( crossempty = log_softmax + nll )
#         preds_logsoft = preds_logsoft.gather(1,labels.view(-1,1))
#         alpha = self.alpha.gather(0,labels.view(-1))
#         loss = -torch.mul(torch.pow((1-preds_softmax), self.gamma), preds_logsoft)  # torch.pow((1-preds_softmax), self.gamma) 为focal loss中 (1-pt)**γ

#         loss = torch.mul(alpha, loss.t())
#         if self.size_average:
#             loss = loss.mean()
#         else:
#             loss = loss.sum()
#         return loss

## (2) Model Architecture

In [170]:
class MyModel(nn.Module):
    def __init__(self, config):
        super(MyModel, self).__init__()
        self.seq_length = config['seq_length']
        self.lstm_1 = nn.LSTM(7, 32, num_layers=2, dropout=0.2, batch_first=True, bidirectional=False)
        # self.lstm_1 = nn.LSTM(7, 16, num_layers=2, batch_first=True, bidirectional=False)
        self.sequential = nn.Sequential(
            nn.Linear(32, 2)
        )
        # self.softmax = nn.Softmax()

    def forward(self, X_2):
        h_tech_list = []
        # for i in range(self.seq_length):
        #   out = X_2[:, i, :]
        #   h_tech_list.append(out)
        # h_tech = torch.stack(h_tech_list, dim=1)  # 5 [64, 500] tensor to 1 [64, 5, 500] tensor
        h_tech = X_2


        # 3. LSTM
        out, _ = self.lstm_1(h_tech)
        out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term

        final_out = self.sequential(out)

        return final_out


## (3) Trainer

In [171]:
def trainer(model, train_loader, val_loader, config, device):
    # criterion = FocalLoss(gamma=2, alpha=1)
    # criterion = focal_loss

    criterion = config['criterion']
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

    writer = SummaryWriter()  # Writer of tensoboard.
    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    # 1. Training
    for epoch in range(n_epochs):
      model.train()  # Set the model to training mode
      loss_record = []

      train_pbar = tqdm(train_loader, position=0, leave=True)  # tqdm is a package to visualize your training progress.
      for X_2, y in train_loader:
        optimizer.zero_grad()  # Set gradient to zero

        # Forward pass
        X_2, y = X_2.to(device), y.to(device)
        pred = model(X_2)
        loss = criterion(pred, y)
        loss.backward()                     # Compute gradient(backpropagation).
        optimizer.step()                    # Update parameters.
        step += 1
        loss_record.append(loss.detach().item())

        # Display current epoch number and loss on tqdm progress bar.
        train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
        train_pbar.set_postfix({'loss': loss.detach().item()})

      mean_train_loss = sum(loss_record)/len(loss_record)
      writer.add_scalar('Loss/train', mean_train_loss, step)

      # 2. Evaluation
      model.eval() # Set your model to evaluation mode.
      loss_record = []
      # correct = 0
      # total = 0
      predicted_labels_list = []
      targets_list = []
      for X_2, y in val_loader:
          X_2, y = X_2.to(device), y.to(device)
          with torch.no_grad():
              pred = model(X_2)
              _, predicted = torch.max(pred, 1)
              loss = criterion(pred, y)
              predicted_labels_list.extend(predicted.tolist())
              targets_list.extend(y.tolist())
              loss_record.append(loss.item())

      accuracy = accuracy_score(targets_list, predicted_labels_list)

      # Mean
      mean_valid_loss = sum(loss_record)/len(loss_record)
      print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}, Val Acc: {accuracy:.4f}')
      writer.add_scalar('Loss/valid', mean_valid_loss, step)

      # 3. Judge of saving model
      if mean_valid_loss < best_loss:
          best_loss = mean_valid_loss
          torch.save(model.state_dict(), config_2['save_path']) # Save your best model
          print('Saving model with loss {:.3f}...'.format(best_loss))
          early_stop_count = 0
      else:
          early_stop_count += 1

      if early_stop_count >= config['early_stop']:
          print('\nModel is not improving, so we halt the training session.')
          return


2. Accuracy-Based Trainer

In [172]:
# def trainer(model, train_loader, val_loader, config, device):

#     criterion = config['criterion']
#     optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

#     writer = SummaryWriter()  # Writer of tensoboard.
#     n_epochs, best_accuracy, step, early_stop_count = config['n_epochs'], 0.0, 0, 0

#     # 1. Training
#     for epoch in range(n_epochs):
#         model.train()  # Set the model to training mode
#         loss_record = []

#         train_pbar = tqdm(train_loader, position=0, leave=True)  # tqdm is a package to visualize your training progress.
#         for X_2, y in train_loader:
#             optimizer.zero_grad()  # Set gradient to zero

#             # Forward pass
#             X_2, y = X_2.to(device), y.to(device)
#             pred = model(X_2)

#             y = y.reshape(-1, 1)
#             loss = criterion(pred, y)
#             loss.backward()                     # Compute gradient(backpropagation).
#             optimizer.step()                    # Update parameters.
#             step += 1
#             loss_record.append(loss.detach().item())

#             train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]') # Display current epoch number and loss on tqdm progress bar.
#             train_pbar.set_postfix({'loss': loss.detach().item()})

#         mean_train_loss = sum(loss_record)/len(loss_record)
#         writer.add_scalar('Loss/train', mean_train_loss, step)

#         # 2. Evaluation
#         model.eval() # Set your model to evaluation mode.
#         correct = 0
#         total = 0

#         for X_2, y in val_loader:
#             X_2, y = X_2.to(device), y.to(device)
#             with torch.no_grad():
#                 pred = model(X_2)
#                 y = y.reshape(-1, 1)
#                 loss = criterion(pred, y)
#                 _, predicted = torch.max(pred, 1)
#                 total += y.size(0)
#                 correct += (predicted == y).sum().item()

#         accuracy = correct / total
#         accuracy = accuracy/config['batch_size']

#         # Print and log accuracy
#         print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Validation Accuracy: {accuracy:.4f}')
#         writer.add_scalar('Accuracy/valid', accuracy, step)

#         # 3. Judge of saving model based on accuracy
#         if accuracy > best_accuracy:
#             best_accuracy = accuracy
#             torch.save(model.state_dict(), config_2['save_path']) # Save your best model
#             print('Saving model with accuracy {:.3f}...'.format(best_accuracy))
#             early_stop_count = 0
#         else:
#             early_stop_count += 1

#         if early_stop_count >= config['early_stop']:
#             print('\nModel is not improving, so we halt the training session.')
#             return


## (4) Load Model

1. Load pretrain model

In [173]:
# 載入預訓練模型
# base_model = BertModel.from_pretrained('bert-base-uncased')
# base_model = PreModel(base_model)
# base_model.load_state_dict(torch.load(config_2['pretrained_model_path']))

# Parameter
# section_length = len(df['section_dummy'][0])
# publication_length = len(df['publication_dummy'][0])
# X_2_length = len(feature) - 6

2-1. Initial Model

In [174]:
model = MyModel(config)
model.to(device)

MyModel(
  (lstm_1): LSTM(7, 32, num_layers=2, batch_first=True, dropout=0.2)
  (sequential): Sequential(
    (0): Linear(in_features=32, out_features=2, bias=True)
  )
)

2-2. Contunue training from model.ckpt

In [175]:
# model = MyModel(base_model, config, section_length, publication_length, X_2_length)
# model.load_state_dict(torch.load(config_2['continue_model_path']))
# model.to(device)

## (5) Require_grad

In [176]:

# # Freeze all layers
# for param in model.pretrained_model.parameters():
#   param.requires_grad = False

# for param in model.pretrained_model.base_model.encoder.layer[11].parameters():
#     param.requires_grad = True

# for param in model.pretrained_model.fc1.parameters():
#     param.requires_grad = True

# # for param in model.fc1.parameters():
# #     param.requires_grad = True

# # Check requires_grad status
# for name, param in model.named_parameters():
#     print(name, param.requires_grad)

In [177]:
# Freeze all layers
# for param in model.base_model.parameters():
#   param.requires_grad = False

# Unfreeze part of layers
# for param in model.base_model.encoder.layer[6].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[7].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[8].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[9].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[10].parameters():
#     param.requires_grad = True

# for param in model.base_model.base_model.encoder.layer[11].parameters():
#     param.requires_grad = True

# for param in model.base_model.fc1.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc2.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc3.parameters():
    # param.requires_grad = True

# Check requires_grad status
for name, param in model.named_parameters():
    print(name, param.requires_grad)

lstm_1.weight_ih_l0 True
lstm_1.weight_hh_l0 True
lstm_1.bias_ih_l0 True
lstm_1.bias_hh_l0 True
lstm_1.weight_ih_l1 True
lstm_1.weight_hh_l1 True
lstm_1.bias_ih_l1 True
lstm_1.bias_hh_l1 True
sequential.0.weight True
sequential.0.bias True


# Training

In [178]:
same_seed(config['seed'])
# 全部訓練
trainer(model, train_loader, val_loader, config, device)

# # 分段訓練
# trainer2(model, train_loader, val_loader, config, device)
# trainer1(model, train_loader, val_loader, config, device)

Epoch [1/3000]:   0%|          | 0/19 [00:00<?, ?it/s, loss=0.696]

Epoch [1/3000]: Train loss: 0.7022, Valid loss: 0.7063, Val Acc: 0.4400
Saving model with loss 0.706...


Epoch [2/3000]:   0%|          | 0/19 [00:00<?, ?it/s, loss=0.695]

Epoch [2/3000]: Train loss: 0.7005, Valid loss: 0.7038, Val Acc: 0.4400
Saving model with loss 0.704...


Epoch [3/3000]:   0%|          | 0/19 [00:00<?, ?it/s, loss=0.695]

...

Epoch [252/3000]: Train loss: 0.6856, Valid loss: 0.6859, Val Acc: 0.5700


Epoch [253/3000]:   0%|          | 0/19 [00:00<?, ?it/s, loss=0.725]

Epoch [253/3000]: Train loss: 0.6859, Valid loss: 0.6860, Val Acc: 0.5650


Epoch [254/3000]:   0%|          | 0/19 [00:00<?, ?it/s, loss=0.727]

Epoch [254/3000]: Train loss: 0.6891, Valid loss: 0.6858, Val Acc: 0.5750

Model is not improving, so we halt the training session.


In [179]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

Reusing TensorBoard on port 6006 (pid 1845), started 0:36:40 ago. (Use '!kill 1845' to kill it.)

<IPython.core.display.Javascript object>

# Evaluate

In [180]:
# Evaluation Dataloader
con_train_loader = DataLoader(train_dataset, batch_size=1, shuffle=config['shuffle'], pin_memory=True)
con_val_loader = DataLoader(val_dataset, batch_size=1, shuffle=config['shuffle'], pin_memory=True)
con_test_loader = DataLoader(test_dataset, batch_size=1, shuffle=config['shuffle'], pin_memory=True)

model = MyModel(config)
model.load_state_dict(torch.load(config_2['save_path']))
model.to(device)

# Evaluation mode
model.eval()

accuracy_list = []

# 1. Train part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for inputs, targets in con_train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(targets.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================================================================================')
print('Training Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list), '\n')

# 2. Val part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for inputs, targets in con_val_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(targets.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)

print('=====================================================')
print('Val Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))


# 3. Test part
with torch.no_grad():
    predicted_labels_list = []
    targets_list = []
    for inputs, targets in con_test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(targets.tolist())

    # 计算准确率
    accuracy = accuracy_score(targets_list, predicted_labels_list)
    accuracy_list.append(accuracy)



print('=====================================================')
print('Testing Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))

print('=====================================================', '\n')
print("Accuracy [Train, Val, Test]: ", accuracy_list, '\n')
print(model, '\n')
print('Config: ', config, '\n')
print('Feature: ', feature)
print('time_start: ', time_start, 'time_end: ', time_end)

Training Result:
              precision    recall  f1-score   support

           0       0.54      0.07      0.13       280
           1       0.54      0.94      0.69       322

    accuracy                           0.54       602
   macro avg       0.54      0.51      0.41       602
weighted avg       0.54      0.54      0.43       602

[[ 21 259]
 [ 18 304]] 

Val Result:
              precision    recall  f1-score   support

           0       0.64      0.20      0.31        88
           1       0.59      0.91      0.72       112

    accuracy                           0.60       200
   macro avg       0.62      0.56      0.51       200
weighted avg       0.61      0.60      0.54       200

[[ 18  70]
 [ 10 102]]
Testing Result:
              precision    recall  f1-score   support

           0       0.53      0.10      0.17        88
           1       0.57      0.93      0.71       113

    accuracy                           0.57       201
   macro avg       0.55      0.52  